<a href="https://colab.research.google.com/github/ChyYasir/Imp_tools_and_scripts/blob/main/Summary_From_Youtube_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Necessary tools and libraries**

In [1]:
!pip install -q yt_dlp
!pip install -q git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install -y ffmpeg
!pip install torch
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 30.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s e

## **Import Libraries**



In [2]:
import yt_dlp
import whisper
from google.colab import files


# **Download Audio from YouTube**

> ***Provide a youtube video url***



In [7]:
def download_audio_and_get_title(youtube_url, output_dir="audio"):
    # Get video info (title)
    ydl_opts_info = {
        'quiet': True,
        'skip_download': True,
        'noplaylist': True
    }

    with yt_dlp.YoutubeDL(ydl_opts_info) as ydl:
        info_dict = ydl.extract_info(youtube_url, download=False)
        title = info_dict.get('title', 'audio').replace(" ", "_").replace("/", "_")

    audio_output_path = f"{output_dir}_{title}"

    # Download audio
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': f'{audio_output_path}.%(ext)s',
        'noplaylist': True,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'quiet': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])

    return f"{audio_output_path}.mp3", title


# Replace with your own video link
youtube_url = "https://www.youtube.com/watch?v=acz_OaV9w7o"

# Download and get title
audio_path, title = download_audio_and_get_title(youtube_url)
print(f"Downloaded: {audio_path}")

Downloaded: audio_Technology_Architect_Course_-_PART_1_-_INTRO_(Technical,_Solutions,_Cloud,_Software_Architect).mp3


# **Transcribe Audio with Whisper**

In [8]:
# Load model and transcribe
model = whisper.load_model("base")  # You can use "small", "medium", or "large" for better accuracy
result = model.transcribe(audio_path)
transcription = result['text']
print(transcription[:1000])  # Preview first part


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Hey guys, how are you doing? Now, there are quite a few people that follow my channel who are experienced developers. In the sense they have more than five and six years of experience. After a while, after you've been programming for a while, you want to start learning things from a macro perspective, from a technology architecture perspective. This is why I'm starting this series where I'm going to be sharing all of my knowledge with regards to technology architecture. And I'll be sharing information from five different books. So I'll be compiling all that information and I'll be basically sharing all that information here in the series. And the series is going to be quite long. In the sense I'll be making these 10, 15 minute videos and but they'll keep continuing for a very long period of time. So until all my content has exhausted. That's because technology architecture is a big subject. And it has a lot of topics, a lot of things to cover. And there's not enough content on this, o

In [9]:
# Save to .txt file using title
transcription_file = f"transcription_{title}.txt"
with open(transcription_file, "w", encoding="utf-8") as f:
    f.write(transcription)

# Download the file
files.download(transcription_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Option A: Using Hugging Face Transformers (free)**

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
chunks = [transcription[i:i+1000] for i in range(0, len(transcription), 1000)]

summary = ""
for chunk in chunks:
    summary_part = summarizer(chunk, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
    summary += summary_part + "\n"

print(summary)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu
Your max_length is set to 130, but your input_length is only 48. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=24)


Learn 101 different concepts in terms related to computer science the cloud and of course Stalker. We'll unlock the power inside this container by learning 101 different Concepts in terms of computer science.
Most software is delivered via the internet through the magic of networking when you watch a YouTube video. Your computer is called a client but you and billions of other users are getting that data from remote computers is called servers. When an app starts reaching Millions of people weird things begin to happen the CPU becomes exhausted handling all the incoming requests Disguise slows down network bandwidth gets maxed out and the database becomes too large to query effectively.
Docker is an open source operating system that lets you run multiple operating systems on a single machine. Applications running on top of the Docker engine all share the same host operating system kernel and use resources dynamically based on their needs.
Docker works in three easy steps first you star

# **For Proper Rendering in Github**

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

import nbformat
import IPython

def clear_colab_widgets():
    notebook = IPython.get_ipython().kernel.notebook
    notebook_path = notebook.get("path", "Untitled.ipynb")
    with open(notebook_path, 'r', encoding='utf-8') as f:
        nb = nbformat.read(f, as_version=4)
    if "widgets" in nb.metadata:
        del nb.metadata["widgets"]
    with open(notebook_path, 'w', encoding='utf-8') as f:
        nbformat.write(nb, f)
    print("Cleared widget metadata successfully!")

# Run this before saving to GitHub
clear_colab_widgets()


AttributeError: 'Kernel' object has no attribute 'notebook'